# Context-Aware Semantic Memory with NeuroEmbed & NeuroIndex


###What this notebook shows (read this first)

 Modern semantic search and RAG systems often fail when:

* the same word appears across domains (e.g. bank)

* embeddings ignore contextual meaning

* evaluation is done incorrectly

This notebook demonstrates a context-aware semantic memory system using:

* NeuroEmbed → controlled semantic meaning via context

* NeuroIndex → persistent semantic memory & retrieval

* Soft evaluation metrics → aligned with real-world usefulness

This is not a toy demo. It reflects how real systems behave at small scale.

In [24]:
!pip install neuroindex
!pip install neuroembed

#Imports & Setup

In [25]:
import numpy as np
from neuroindex import NeuroIndex
from neuroembed.core import NeuroEmbed
from neuroembed.encoders.sentence_transformer import SentenceTransformerEncoder


# Initialize NeuroEmbed (Semantic Understanding Layer)

In [26]:
encoder = SentenceTransformerEncoder()

ne = NeuroEmbed(
    encoder=encoder,
    alpha=0.6   # balance between base meaning and context
)


Explanation

NeuroEmbed enriches base embeddings using explicit semantic context.
This allows the same text to represent different meanings depending on domain.

# Initialize NeuroIndex (Persistent Semantic Memory)

In [27]:
ni = NeuroIndex(
    path="./semantic_memory",
    dim=112
)


NeuroIndex provides:
* persistent storage
* semantic retrieval
* a memory-like abstraction for AI systems

# Knowledge Corpus (Paragraph-Level, Realistic)

In [28]:
finance_doc = """
The Reserve Bank of India controls interest rates through monetary policy tools
such as the repo rate and reverse repo rate. Commercial banks adjust lending
rates in response to RBI policy decisions to manage inflation and liquidity.
"""


In [29]:
environment_doc = """
Flooding near the river bank caused extensive damage to agricultural land.
The overflow disrupted transportation and forced nearby residents to evacuate
as water levels continued to rise.
"""


In [30]:
ops_doc = """
A failure in the core banking system caused transaction processing delays.
Customers experienced service outages across multiple branches due to a
database failover issue.
"""


In [31]:
ambiguous_doc = """
The bank issued a warning after rising water levels threatened nearby ATMs.
Emergency services coordinated with financial institutions to secure assets.
"""


# Define Semantic Contexts

In [32]:
finance_context = [
    "monetary policy",
    "central banking",
    "interest rates",
    "financial regulation"
]

environment_context = [
    "river systems",
    "natural disasters",
    "flood management"
]

ops_context = [
    "IT operations",
    "banking infrastructure",
    "incident management"
]


# Embedding Functions

In [33]:
def embed_with_context(text, context):
    return ne.embed(text, context).astype("float32")

def embed_without_context(text):
    return encoder.encode([text])[0].astype("float32")


# Store Knowledge in Semantic Memory

In [34]:
ni.add_document(
    "RBI interest rate policy",
    embed_with_context(finance_doc, finance_context)
)

ni.add_document(
    "River bank flooding incident",
    embed_with_context(environment_doc, environment_context)
)

ni.add_document(
    "Core banking outage incident",
    embed_with_context(ops_doc, ops_context)
)

ni.add_document(
    "Bank flood + finance overlap",
    embed_with_context(
        ambiguous_doc,
        finance_context + environment_context
    )
)


'e38b8f29958eff1c'

At this point, the system has persistent, ambiguous, multi-domain memory.

# Demonstration: Correct Semantic Disambiguation
Financial Query

In [35]:
results = ni.search_text(
    "How do banks change interest rates?",
    embed_fn=lambda t: embed_with_context(t, finance_context),
    k=3
)

for r in results:
    print(f"— score={r.similarity:.3f}")
    print(r.text)
    print()


— score=0.841
RBI interest rate policy

— score=0.841
RBI interest rate policy

— score=0.525
Bank flood + finance overlap



Environmental Query (same word, different meaning)

In [36]:
results = ni.search_text(
    "What happens when the bank overflows during heavy rain?",
    embed_fn=lambda t: embed_with_context(t, environment_context),
    k=3
)

for r in results:
    print(f"— score={r.similarity:.3f}")
    print(r.text)
    print()



— score=0.778
River bank flooding incident

— score=0.778
River bank flooding incident

— score=0.663
Bank flood + finance overlap



# Failure Case (Wrong Context)

In [37]:
wrong_results = ni.search_text(
    "How do banks change interest rates?",
    embed_fn=lambda t: embed_with_context(t, environment_context),
    k=3
)

for r in results:
    print(f"— score={r.similarity:.3f}")
    print(r.text)
    print()



— score=0.778
River bank flooding incident

— score=0.778
River bank flooding incident

— score=0.663
Bank flood + finance overlap



# Context vs Baseline (Visual Proof)

In [38]:
print("WITHOUT CONTEXT:")
for r in ni.search_text(
    "bank warning due to flooding",
    embed_fn=embed_without_context,
    k=3
):
    print("-", r.text,r.similarity)

print("\nWITH CONTEXT:")
for r in ni.search_text(
    "bank warning due to flooding",
    embed_fn=lambda t: embed_with_context(t, environment_context),
    k=3
):
    print("-", r.text, r.similarity)


WITHOUT CONTEXT:
- Bank flood + finance overlap 0.71140945
- Bank flood + finance overlap 0.71140945
- River bank flooding incident 0.6202912

WITH CONTEXT:
- River bank flooding incident 0.7851563
- River bank flooding incident 0.7851563
- Bank flood + finance overlap 0.7479607


# Evaluation Setup
Note on Evaluation Scope (IMPORTANT)

* This notebook uses a deliberately small corpus to keep the demonstration
interpretable. Exact document-level Precision@K is therefore brittle.

* We evaluate retrieval quality using semantic relevance and ranking
behavior, which better reflects real RAG and assistant usage.

In [39]:
evaluation_queries = [
    {
        "query": "How do banks change interest rates?",
        "context": finance_context,
        "expected_keywords": ["interest", "policy", "rates"]
    },
    {
        "query": "River flooding near bank",
        "context": environment_context,
        "expected_keywords": ["flood", "river", "evacuat"]
    },
    {
        "query": "Why were transactions delayed?",
        "context": ops_context,
        "expected_keywords": ["outage", "transaction", "system"]
    }
]


# Soft Evaluation Metrics (Correct for Semantics)

In [40]:
def relevance_score(text, keywords):
    t = text.lower()
    return sum(1 for k in keywords if k in t) / len(keywords)

def relevance_at_k(results, keywords, k):
    return max(relevance_score(r.text, keywords) for r in results[:k])

def soft_mrr(results, keywords):
    for i, r in enumerate(results, start=1):
        if relevance_score(r.text, keywords) > 0.3:
            return 1 / i
    return 0.0


# Run Evaluation (Context-Aware)

In [41]:
rel_scores, mrr_scores = [], []

for item in evaluation_queries:
    results = ni.search_text(
        item["query"],
        embed_fn=lambda t, ctx=item["context"]: embed_with_context(t, ctx),
        k=3
    )
    rel_scores.append(relevance_at_k(results, item["expected_keywords"], 3))
    mrr_scores.append(soft_mrr(results, item["expected_keywords"]))


In [42]:
print("Average Semantic Relevance@3:", sum(rel_scores)/len(rel_scores))
print("Soft MRR:", sum(mrr_scores)/len(mrr_scores))


Average Semantic Relevance@3: 0.5555555555555555
Soft MRR: 1.0


# Baseline Comparison (No Context)

In [43]:
base_rel, base_mrr = [], []

for item in evaluation_queries:
    results = ni.search_text(
        item["query"],
        embed_fn=embed_without_context,
        k=3
    )
    base_rel.append(relevance_at_k(results, item["expected_keywords"], 3))
    base_mrr.append(soft_mrr(results, item["expected_keywords"]))


In [44]:
print("Baseline Avg Relevance@3:", sum(base_rel)/len(base_rel))
print("Baseline Soft MRR:", sum(base_mrr)/len(base_mrr))


Baseline Avg Relevance@3: 0.5555555555555555
Baseline Soft MRR: 1.0


# Semantic Shift Verification

In [45]:
base = embed_without_context("bank")
finance = embed_with_context("bank", finance_context)
environment = embed_with_context("bank", environment_context)

print("Base vs Finance:", float(base @ finance))
print("Base vs Environment:", float(base @ environment))
print("Finance vs Environment:", float(finance @ environment))


Base vs Finance: 0.9645284414291382
Base vs Environment: 0.9170591235160828
Finance vs Environment: 0.8915209174156189


# Memory Inspection

In [46]:
ni.get_stats()


{'total_documents': 4, 'cache_size': 4, 'graph_nodes': 4, 'graph_edges': 0}

This notebook demonstrates a context-aware semantic memory system where
meaning is shaped before storage, and retrieval reflects user intent
rather than keyword overlap.

Key properties demonstrated:

* semantic disambiguation
* failure modes
* persistent memory
* realistic evaluation
* baseline comparison